In [1]:
import re
import pandas as pd


from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize



## TF-IDF VECTORIZER ##

In [2]:
df_train = pd.read_csv('train.csv', on_bad_lines='skip', sep=chr(1))
df_test = pd.read_csv('test.csv', on_bad_lines='skip', sep=chr(1))


/var/folders/q_/fk7mrt1x27g0xm9pkch74j5m0000gn/T/ipykernel_78411/888051889.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df_test = pd.read_csv('test.csv', on_bad_lines='skip', sep=chr(1))


In [3]:
df_train.head()

,product_id,product_text,category
0,214477867,Lg K11 2.4a Hızlı Şarj Kablosu 2m Golden Serie...,Şarj Cihazı & Kablo
1,104723304,Müthiş Satisfyer Dual Pleasure Telefon Kontrol...,Vibratör
2,134486860,Sıva Altı Dekoratif 12w Modül Led 3000k(GÜNIŞI...,Spot Lamba
3,106247966,Pc Ekmek Kutusu,Ekmek Kutusu ve Sepetleri
4,117703257,3mm Philips 55pus6262 Uyumlu Tv Ekran Koruyucu...,TV Ekran Koruyucular


In [4]:
df_test.head()

,product_id,product_text,category
0,206870614,Marka: 18214 Disney Princess Ahşap Puzzle 16 P...,Spor Oyuncakları
1,44839736,40507 Ortopedik Günlük Erkek Terlik,Terlik
2,36998202,Kadın Beyaz 3 Çift Pamuklu Bilek Boyu Çorap,Spor Çorap
3,6560978,Bebek Bakım Kremi 250 gr 2 Adet,Bebek Kremi ve Yağı
4,138707066,Fırsat Ayı Krom 5 Katlı Pantolon Eşarp Askısı ...,Pantolon Askısı


In [5]:
df_train = df_train.dropna()
df_train['category'].value_counts()


Bagaj Havuzu               52899
Bot & Bootie               49106
Bijuteri Küpe              44818
Omuz Çantası               43738
Gömlek                     40909
                           ...  
Endüstriyel Temizleme        228
Paketleme Malzemeleri        222
Piknik Malzemesi             214
Çocuk Odası Düzenleyici      187
Gazoz                        132
Name: category, Length: 1140, dtype: int64

In [6]:
# Calculate sample size for each category
category_counts = df_train['category'].value_counts()

# Filter categories with sample size > 500
categories_above_threshold = category_counts[category_counts < 5000].index.tolist()

# Filter the DataFrame based on categories above threshold
df_filtered = df_train[df_train['category'].isin(categories_above_threshold)]

category_counts = df_filtered['category'].value_counts()

print("Category Sample Counts:")
print(category_counts)

Category Sample Counts:
Büyük Beden Elbise         4985
El Feneri                  4973
Yüz Temizleyici            4863
Masa Lambası               4863
Sandalye Minderi           4832
                           ... 
Endüstriyel Temizleme       228
Paketleme Malzemeleri       222
Piknik Malzemesi            214
Çocuk Odası Düzenleyici     187
Gazoz                       132
Name: category, Length: 828, dtype: int64


In [7]:
df_filtered_train = df_train[(df_train['category'] == 'Yüz Temizleyici') | (df_train['category'] == 'Sandalye Minderi')| (df_train['category'] == 'Büyük Beden Elbise')| (df_train['category'] == 'El Feneri')| (df_train['category'] == 'Masa Lambası')]
df_filtered_train = df_filtered_train.reset_index(drop=True)

df_filtered_train

,product_id,product_text,category
0,123354169,Pofidik Kare Sandalye Minderi 40x40 Cm,Sandalye Minderi
1,6583505,Özel Tasarım Fermuarlı Sandalye Minderi,Sandalye Minderi
2,35322790,Rattan Koltuk Minderi Su Geçirmez Kumaş 5 cm G...,Sandalye Minderi
3,69564220,Temizleyici Yüz Toniği 150 Ml,Yüz Temizleyici
4,95315892,Gp-209 Şarjlı Mıknatıslı Kancalı Döndürülebili...,El Feneri
...,...,...,...
24511,154298872,Masaüstü Kıstırmalı Lamba,Masa Lambası
24512,162681631,El Feneri 519,El Feneri
24513,189736850,Gold Silver Gs-530 Usb Ile Şarj Edilebilir T6 ...,El Feneri
24514,180075017,F.b.ı F-82 T6 Led Zoomlu 2000 Lm Yüksek Işık Ş...,El Feneri


In [8]:
df_filtered_test = df_test[(df_test['category'] == 'Yüz Temizleyici') | (df_test['category'] == 'Sandalye Minderi')| (df_test['category'] == 'Büyük Beden Elbise')| (df_test['category'] == 'El Feneri')| (df_test['category'] == 'Masa Lambası')]
df_filtered_test

,product_id,product_text,category
445,74908572,Dermapen Iğnesi (15 Adet - 12 Iğneli) Ultima A...,Yüz Temizleyici
486,213609159,Yılbaşı Temalı Soft Kırmızı Gül Tasarımlı 4 Yu...,Sandalye Minderi
546,105825089,İskandinav Sandalye Minderi,Sandalye Minderi
927,129872878,Kadın Lacivert Etek Ucu Dantel Kolsuz Viskon E...,Büyük Beden Elbise
1155,75471988,Kadın Kolsuz Parti Kokteyl Elbisesi,Büyük Beden Elbise
...,...,...,...
1714069,213614162,Kırmızı Ekoseli Geyik Tasarımlı 4 Yuvarlak Pof...,Sandalye Minderi
1714386,80693435,Tokyo Ghoul Ken Kaneki 16 Renk Gece Lambası,Masa Lambası
1714491,209122149,Ekose Detaylı Çift Ip Elbise,Büyük Beden Elbise
1714516,38869966,Dijital Baskılı Pofidik Sandalye Minderi,Sandalye Minderi


In [9]:
# Get Turkish stop words
turkish_stop_words = set(stopwords.words('turkish'))

# Function to remove stop words from text
def remove_stop_words(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    tokens = word_tokenize(text)
    filtered_tokens = [word for word in tokens if word.lower() not in turkish_stop_words]
    return ' '.join(filtered_tokens)
# Remove stop words from the 'text' column
df_filtered_train['product_text'] = df_filtered_train['product_text'].apply(remove_stop_words)
df_filtered_test['product_text'] = df_filtered_test['product_text'].apply(remove_stop_words)

/var/folders/q_/fk7mrt1x27g0xm9pkch74j5m0000gn/T/ipykernel_78411/159079942.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered_test['product_text'] = df_filtered_test['product_text'].apply(remove_stop_words)


In [10]:

label_encoder = LabelEncoder()

y_train = label_encoder.fit_transform(df_filtered_train['category'])
y_test = label_encoder.transform(df_filtered_test['category'])

# Create the RandomUnderSampler object
rus = RandomUnderSampler(random_state=42)

# Perform undersampling on the data
X_train_resampled, y_train_resampled = rus.fit_resample(df_filtered_train, y_train)

X_train_resampled.head()

,product_id,product_text,category
7346,207302615,büyük beden uzun kollu kuşaklı simli uzun elbi...,Büyük Beden Elbise
13535,105855896,kadın bordo büyük beden uzun şifon astarlı elbise,Büyük Beden Elbise
2241,88207886,kadın bordo çiçek desenli kısa kol elbise,Büyük Beden Elbise
12239,65910593,kadın siyah fırfırlı elbise,Büyük Beden Elbise
3251,192793589,kadın lacivert hakim yaka etnik nakış desenli ...,Büyük Beden Elbise


In [11]:
X_train_resampled

,product_id,product_text,category
7346,207302615,büyük beden uzun kollu kuşaklı simli uzun elbi...,Büyük Beden Elbise
13535,105855896,kadın bordo büyük beden uzun şifon astarlı elbise,Büyük Beden Elbise
2241,88207886,kadın bordo çiçek desenli kısa kol elbise,Büyük Beden Elbise
12239,65910593,kadın siyah fırfırlı elbise,Büyük Beden Elbise
3251,192793589,kadın lacivert hakim yaka etnik nakış desenli ...,Büyük Beden Elbise
...,...,...,...
6890,213063788,gül kürü yüz temizleme köpüğü 150 ml,Yüz Temizleyici
19442,212546135,marka gül kürü yüz temizleme köpüğü 150 ml kat...,Yüz Temizleyici
10953,83799038,clarify salicylic acid foaming cleanser salisi...,Yüz Temizleyici
5340,141777646,semina microblading kalıcı makyaj temizleme kö...,Yüz Temizleyici


In [12]:

# TF-IDF vectorization
vectorizer = TfidfVectorizer()
x_train = vectorizer.fit_transform(X_train_resampled['product_text'].to_list())
x_test = vectorizer.transform(df_filtered_test['product_text'].to_list())

In [13]:
x_train.shape

(24160, 11437)

In [14]:
logreg = LogisticRegression()

# Fit the logistic regression model on train data
logreg.fit(x_train, y_train_resampled)



LogisticRegression()

In [15]:
# Predict labels for test data
predictions = logreg.predict(x_test)

# Calculate accuracy on test data
accuracy = accuracy_score(y_test, predictions)  # Replace [0, 1] with actual labels for test data
print("Accuracy:", accuracy)

Accuracy: 0.9954382535027696


In [16]:
a = vectorizer.transform(["Siyah renkli orta boy aydınlatma"])
logreg.predict(a)

array([2])

In [17]:
a = vectorizer.transform(["Ortopedik"])
logreg.predict(a)

array([3])

In [18]:
dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))

{'Büyük Beden Elbise': 0,
 'El Feneri': 1,
 'Masa Lambası': 2,
 'Sandalye Minderi': 3,
 'Yüz Temizleyici': 4}

In [19]:
df_train_imdb = pd.read_csv('IMDB.csv', on_bad_lines='skip')


In [20]:
df_train_imdb

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [21]:
df_train_imdb = df_train_imdb.dropna()

df_train_imdb['sentiment'].value_counts()


positive    25000
negative    25000
Name: sentiment, dtype: int64

In [22]:
# Get Turkish stop words
english_stop_words = set(stopwords.words('english'))

# Function to remove stop words from text
def remove_stop_words(text):
    text = text.lower() 
    text = re.sub(r'[^\w\s]', '', text)
    tokens = word_tokenize(text)
    filtered_tokens = [word for word in tokens if word.lower() not in english_stop_words]
    return ' '.join(filtered_tokens)
# Remove stop words from the 'text' column
df_train_imdb['review'] = df_train_imdb['review'].apply(remove_stop_words)


In [23]:
df_train_imdb

,review,sentiment
0,one reviewers mentioned watching 1 oz episode ...,positive
1,wonderful little production br br filming tech...,positive
2,thought wonderful way spend time hot summer we...,positive
3,basically theres family little boy jake thinks...,negative
4,petter matteis love time money visually stunni...,positive
...,...,...
49995,thought movie right good job wasnt creative or...,positive
49996,bad plot bad dialogue bad acting idiotic direc...,negative
49997,catholic taught parochial elementary schools n...,negative
49998,im going disagree previous comment side maltin...,negative


In [24]:
label_encoder = LabelEncoder()
y_numeric = label_encoder.fit_transform(df_train_imdb['sentiment'])
X_train, X_test, y_train, y_test = train_test_split(df_train_imdb, y_numeric, test_size=0.25, random_state=42, stratify=y_numeric)



In [25]:
vectorizer = TfidfVectorizer()
x_train = vectorizer.fit_transform(X_train['review'].to_list())
x_test = vectorizer.transform(X_test['review'].to_list())

In [26]:
logreg = LogisticRegression()

# Fit the logistic regression model on train data
logreg.fit(x_train, y_train)



LogisticRegression()

In [27]:
# Predict labels for test data
predictions = logreg.predict(x_test)

# Calculate accuracy on test data
accuracy = accuracy_score(y_test, predictions)  # Replace [0, 1] with actual labels for test data
print("Accuracy:", accuracy)

Accuracy: 0.89632
